This Python script trains a YOLO (You Only Look Once) model on the COCO dataset using PyTorch. The script sets up the model, loads the COCO dataset, defines the training loop, and logs the results.

## Loading Dataset in google colab from gdrive

In [ ]:
!gdown --folder --id 1B19RWODijlUlXEdvLKxjcmlg-BiJcbWE -O /content/shared_folder

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Retrieving folder contents
Processing file 1tytlWWD1IqtyH3UY1cNDe5y00QJwuGh7 coco2017_subset.zip
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1tytlWWD1IqtyH3UY1cNDe5y00QJwuGh7
From (redirected): https://drive.google.com/uc?id=1tytlWWD1IqtyH3UY1cNDe5y00QJwuGh7&confirm=t&uuid=4648732c-8bda-4a34-a185-b50305f9bf6e
To: /content/shared_folder/coco2017_subset.zip
100% 6.71G/6.71G [01:58<00:00, 56.5MB/s]
Download completed


In [ ]:
import os
os.listdir("/content/shared_folder")

['coco2017_subset.zip']

In [ ]:
!unzip /content/shared_folder/coco2017_subset.zip -d /content/extracted_files

Streaming output truncated to the last 5000 lines.
  inflating: /content/extracted_files/coco2017_subset/train2017/000000508801.jpg  
  inflating: /content/extracted_files/coco2017_subset/train2017/000000508836.jpg  
  inflating: /content/extracted_files/coco2017_subset/train2017/000000508855.jpg  
  inflating: /content/extracted_files/coco2017_subset/train2017/000000508861.jpg  
  inflating: /content/extracted_files/coco2017_subset/train2017/000000508878.jpg  
  inflating: /content/extracted_files/coco2017_subset/train2017/000000508881.jpg  
  inflating: /content/extracted_files/coco2017_subset/train2017/000000508906.jpg  
  inflating: /content/extracted_files/coco2017_subset/train2017/000000508938.jpg  
  inflating: /content/extracted_files/coco2017_subset/train2017/000000508969.jpg  
  inflating: /content/extracted_files/coco2017_subset/train2017/000000508972.jpg  
  inflating: /content/extracted_files/coco2017_subset/train2017/000000508984.jpg  
  inflating: /content/extracted_file

## Importing required libraries

In [ ]:
import os
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import pandas as pd
import cv2
import seaborn as sns
import random
from skimage import io
import albumentations as A
import json

## Annotation generation for Train data

In [ ]:
# Path to COCO annotation file and output folder for YOLO labels
coco_ann_file = "/content/extracted_files/coco2017_subset/annotation_subset/instances_train2017_subset.json"
output_dir = "/content/extracted_files/coco2017_subset/train2017_labels"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Load COCO annotations
with open(coco_ann_file, 'r') as f:
    coco_data = json.load(f)

In [ ]:
# Create mapping: COCO category id -> new id (0-based contiguous indices)
# For example, if valid categories are 1, 2, 3, 5, 6, ... then you might have:
coco_categories = sorted([cat['id'] for cat in coco_data['categories']])
id_mapping = {orig_id: new_id for new_id, orig_id in enumerate(coco_categories)}

In [ ]:
# Build a mapping from image_id to image info
img_info = {img['id']: img for img in coco_data['images']}

In [ ]:
# Process annotations per image
annotations_by_image = {}
for ann in coco_data['annotations']:
    img_id = ann['image_id']
    img = img_info[img_id]
    img_width, img_height = img['width'], img['height']

    x, y, w, h = ann['bbox']
    # Convert COCO bbox [x, y, w, h] to YOLO format [x_center, y_center, width, height] normalized
    x_center = (x + w / 2) / img_width
    y_center = (y + h / 2) / img_height
    norm_w = w / img_width
    norm_h = h / img_height

    # Remap the category id
    orig_cat = ann['category_id']
    if orig_cat not in id_mapping:
        continue  # skip if the category is not in the mapping
    new_cat = id_mapping[orig_cat]

    if img_id not in annotations_by_image:
        annotations_by_image[img_id] = []
    annotations_by_image[img_id].append(f"{new_cat} {x_center:.6f} {y_center:.6f} {norm_w:.6f} {norm_h:.6f}")

In [ ]:
# Write YOLO label files for each image
for img_id, ann_list in annotations_by_image.items():
    file_name = img_info[img_id]['file_name']
    base_name = os.path.splitext(file_name)[0]
    # Save the label file in the same directory as your images, or wherever you prefer
    out_file = os.path.join(output_dir, base_name + '.txt')
    with open(out_file, 'w') as f:
        for line in ann_list:
            f.write(line + '\n')

In [ ]:
import shutil

# Define source and destination directories
source_dir = "/content/extracted_files/coco2017_subset/train2017_labels"
destination_dir = "/content/extracted_files/coco2017_subset/train2017"

# Ensure the destination directory exists
os.makedirs(destination_dir, exist_ok=True)

# Move all files from train2017_labels to train2017
for file_name in os.listdir(source_dir):
    shutil.move(os.path.join(source_dir, file_name), destination_dir)

print("Files moved successfully!")

Files moved successfully!


## Annotation generation for Validation data

In [ ]:
# Path to COCO annotation file and output folder for YOLO labels
coco_ann_file = "/content/extracted_files/coco2017_subset/annotation_subset/instances_val2017_subset.json"
output_dir = "/content/extracted_files/coco2017_subset/val2017_labels"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Load COCO annotations
with open(coco_ann_file, 'r') as f:
    coco_data = json.load(f)

In [ ]:
# Build a mapping from image_id to image info
img_info = {img['id']: img for img in coco_data['images']}

In [ ]:
# Process annotations per image
annotations_by_image = {}
for ann in coco_data['annotations']:
    img_id = ann['image_id']
    img = img_info[img_id]
    img_width, img_height = img['width'], img['height']

    x, y, w, h = ann['bbox']
    # Convert COCO bbox [x, y, w, h] to YOLO format [x_center, y_center, width, height] normalized
    x_center = (x + w / 2) / img_width
    y_center = (y + h / 2) / img_height
    norm_w = w / img_width
    norm_h = h / img_height

    # Remap the category id
    orig_cat = ann['category_id']
    if orig_cat not in id_mapping:
        continue  # skip if the category is not in the mapping
    new_cat = id_mapping[orig_cat]

    if img_id not in annotations_by_image:
        annotations_by_image[img_id] = []
    annotations_by_image[img_id].append(f"{new_cat} {x_center:.6f} {y_center:.6f} {norm_w:.6f} {norm_h:.6f}")

In [ ]:
# Write YOLO label files for each image
for img_id, ann_list in annotations_by_image.items():
    file_name = img_info[img_id]['file_name']
    base_name = os.path.splitext(file_name)[0]
    # Save the label file in the same directory as your images, or wherever you prefer
    out_file = os.path.join(output_dir, base_name + '.txt')
    with open(out_file, 'w') as f:
        for line in ann_list:
            f.write(line + '\n')

In [ ]:
# Define source and destination directories
source_dir = "/content/extracted_files/coco2017_subset/val2017_labels"
destination_dir = "/content/extracted_files/coco2017_subset/val2017"

# Ensure the destination directory exists
os.makedirs(destination_dir, exist_ok=True)

# Move all files from train2017_labels to train2017
for file_name in os.listdir(source_dir):
    shutil.move(os.path.join(source_dir, file_name), destination_dir)

print("Files moved successfully!")

Files moved successfully!


## Creation of yaml file

In [ ]:
import yaml

# .yaml configuration
data = {
    'path': "/content/extracted_files/coco2017_subset",    # Root directory
    'val': "/content/extracted_files/coco2017_subset/val2017",                # Path to val images relative to root
    'test': "/content/extracted_files/coco2017_subset/test2017",              # Path to test images relative to root
    'train': "/content/extracted_files/coco2017_subset/train2017",
    'nc': 1,                         # Number of classes (update this later if needed)
    'names': []                      # To be filled with class names
}

# Save the .yaml file in cocodataset folder
with open("/content/extracted_files/coco2017_subset/coco.yaml", 'w') as file:
    yaml.dump(data, file, default_flow_style=False)

print("coco.yaml created successfully!")

coco.yaml created successfully!


In [ ]:
# Load COCO annotations
with open("/content/extracted_files/coco2017_subset/annotation_subset/instances_val2017_subset.json") as f:
    coco_data = json.load(f)

# Extract class names
class_names = [category['name'] for category in coco_data['categories']]
num_classes = len(class_names)

print("Classes:", class_names)
print("Total Classes:", num_classes)

Classes: ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
Total Classes: 80


In [ ]:
data['nc'] = num_classes      # Update number of classes
data['names'] = class_names   # Update class names

# Save updated .yaml
with open("/content/extracted_files/coco2017_subset/coco.yaml", 'w') as file:
    yaml.dump(data, file, default_flow_style=False)

print("coco.yaml updated with classes!")

coco.yaml updated with classes!


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.1/949.1 kB 36.0 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
import os

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)

True
12.4


In [ ]:
# 1.Model Initialization
model = YOLO('yolo11n.pt')

100%|██████████| 5.35M/5.35M [00:00<00:00, 273MB/s]


In [ ]:
# 2.Training Configuration
data_yaml = '/content/extracted_files/coco2017_subset/coco.yaml'

epochs = 3
img_size = 640
batch_size = 1

In [27]:
# 3.Training
print("Starting YOLOv8 training...")
results = model.train(data=data_yaml, epochs=epochs, imgsz=img_size, batch=batch_size, device=0)
print("Training completed.")

Starting YOLOv8 training...
Ultralytics 8.3.90 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/extracted_files/coco2017_subset/coco.yaml, epochs=3, time=None, patience=100, batch=1, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=Fals

100%|██████████| 755k/755k [00:00<00:00, 104MB/s]



                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

train: Scanning /content/extracted_files/coco2017_subset/train2017... 29315 images, 256 backgrounds, 0 corrupt: 100%|██████████| 29571/29571 [00:51<00:00, 573.55it/s] 


train: New cache created: /content/extracted_files/coco2017_subset/train2017.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

                   all       1250       9236       0.55      0.428      0.453      0.307

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        3/3      0.42G      1.286      2.038      1.377         14        640: 100%|██████████| 29571/29571 [48:10<00:00, 10.23it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 625/625 [00:30<00:00, 20.25it/s]


                   all       1250       9236      0.546       0.43      0.453      0.309

3 epochs completed in 2.456 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.90 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 625/625 [00:20<00:00, 30.85it/s]


                   all       1250       9236      0.581      0.434      0.473      0.325
                person        670       2607      0.832      0.551      0.692      0.459
               bicycle         34         75      0.481      0.333      0.334      0.218
                   car        132        457       0.72      0.411      0.505       0.29
            motorcycle         36         88      0.592      0.545      0.556      0.351
              airplane         23         43      0.667      0.837      0.804      0.601
                   bus         43         55      0.683      0.746      0.777      0.653
                 train         48         60      0.803      0.717      0.792      0.587
                 truck         56         96      0.533      0.302       0.33      0.228
                  boat         30        107      0.586      0.172      0.311      0.164
         traffic light         53        176      0.517      0.403      0.387      0.202
          fire hydran

In [28]:
from google.colab import files
files.download("/content/runs/detect/train/weights/best.pt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>